<img src = "pandas_logo.png">

# <center> Week 3 - Data Manipulation with Pandas</center>

<b>Lemma</b> Every finite set contains its supremum.

Proof: 

+ For the case $A=\{a\}$:

    $A = \{a\}$ is a singleton set, then clearly $\sup A = \max A = a$ .

    Next, assume we know that all sets of cardinality $m - 1$ have a maximal element.


+ Then for the case $1 < |A| < m$:  

    Let $a \in A$ be an arbitrary element and let $A^\prime = A - \{a\}$. 

    Since $A^\prime$ has $m-1$ elements, it has a maximum, denoted by $\max A^\prime = a^\prime$. 

    Then $\max\{a,a^\prime\}$ is the maximal element of $A$.



<b>Proposition</b> EC3389 has a most boring class.

Proof: Follows from above since the set of all EC3389 classes is finite.

<b>Conjecture</b> This is it.

<hr>

In [276]:
import pandas as pd # Our new friend
import numpy as np
%matplotlib inline

## Series and DataFrames

A Series is a <i>one</i>-dimensional array-like object containing:

+ Index column(s) labeling data

+ Data column

<b>Creating Series from Lists and NumPy Arrays</b>

In [585]:
idx = ["a", "b", "c", "d", "e"]                            # Some index 

In [586]:
dt = np.random.uniform(low = 0, high = 10, size = (5,))    # Some 1-dimensional data 
s = pd.Series(data = dt, index = idx)

In [587]:
dt = np.random.uniform(low = 0, high = 10, size = (5,4))    # Some multiple dimensional data 
cols = ["col1", "col2", "col3", "col4"]
df = pd.DataFrame(data = dt, index = idx, columns = cols)

Alternatively, you can create a DataFrame using dictionaries.

<b>Attributes and Functions</b>

Series and DataFrames come with <i>many</i> useful attributes and built-in functions, several of them inherited from NumPy. Let's take a look at some of them.

Attributes

+ values: NumPy Array

+ index: a pandas Index object

Functions

+ head()

+ tail()

+ mean()

+ std()

+ var()

+ describe()

+ plot()

<font color ="navy"><b>Index objects</b></font> Index objects also have methods such as append, drop, insert, delete.

<b>Pandas + NumPy</b>

Because Series and DataFrames are array-like, you can treat them almost like NumPy arrays.

In [32]:
np.log(df)  # FAST elementwise application through NumPy (and in turn through C)

col1      col2      col3      col4
a  1.685340  0.205083  1.969622  0.294122
b  1.665199  2.145724  1.708712  1.604618
c -1.527235  1.125705  0.783348  2.255463
d  0.637932  2.117035  0.569742  0.181750
e  1.214424  1.458671  2.190931  2.085142

<b>Reading files a DataFrame</b>

Pandas has can read from several different kinds of files. Try autocompleting from pd.read_...

In [104]:
df = pd.read_csv("baby_names_2014.csv")

<hr>

## Indexing and selection

### Selecting subsets of a Series

In [205]:
idx = ["a", "b", "c", "d", "a"] #  Now 'a' appears twice
dt = np.random.uniform(low = 0, high = 10, size = (5,))
s = pd.Series(data = dt, index = idx)
s

a    2.482175
b    9.612631
c    2.919912
d    8.429130
a    7.688004
dtype: float64

You can select a subset of a Series by calling the index label

In [136]:
s["b"] 

1.359400455402775

By calling the index position

In [137]:
s[2]  # This could also be a slice m:n

0.090844068436640724

By using a list (which returns another Series)

In [139]:
s[["a","b"]]

a    6.621368
a    2.707800
b    1.359400
dtype: float64

<font color = "navy">Or by boolean indexing</font>

In [141]:
# First check out what s > 2 does.
s[s > 2] # Returns a Series containing elements that satisfy the condition

a    6.621368
d    8.958730
a    2.707800
dtype: float64

### Selecting subsets of a DataFrame

In [157]:
dt = np.random.uniform(low = 0, high = 10, size = (5,4))    # Some multiple dimensional data 
cols = ["col1", "col2", "col3", "col4"]
idx = ["a","b","c","d","a"]
df = pd.DataFrame(data = dt, index = idx, columns = cols)

In [167]:
df

col1      col2      col3      col4
a  6.395079  9.547660  6.128023  7.711459
b  2.309072  9.043237  8.875507  4.386701
c  6.318027  8.131001  8.803683  9.228691
d  5.103621  7.207440  7.022683  7.809002
a  7.035591  9.657976  1.477171  8.285552

<b>Using ".ix"</b>

To select by label, use the ".ix" method

In [160]:
df.ix["a"]

col1      col2      col3      col4
a  6.395079  9.547660  6.128023  7.711459
a  7.035591  9.657976  1.477171  8.285552

You can also restrict the columns

In [165]:
df.ix["a", ["col2"]]  # If "col2" not enclosed in a list, it returns a Series

col2
a  9.547660
a  9.657976

Call by position

In [188]:
df.ix["a", ["col1", "col2"]]  

col1      col2
a  6.395079  9.547660
a  7.035591  9.657976

All rows of a particular column

In [170]:
df.ix[:, "col1"]

a    6.395079
b    2.309072
c    6.318027
d    5.103621
a    7.035591
Name: col1, dtype: float64

The method <i>.loc</i> works similarly to <i>.ix</i> but only on the labels

In [192]:
df.loc["a"]

col1      col2      col3      col4
a  6.395079  9.547660  6.128023  7.711459
a  7.035591  9.657976  1.477171  8.285552

The method <i>.iloc</i> works similarly to <i>.ix</i> but only on the positions (integers)

In [193]:
df.iloc[0]

col1    6.395079
col2    9.547660
col3    6.128023
col4    7.711459
Name: a, dtype: float64

<b>Note</b> Some people would argue that you should use "loc" and "iloc", and avoid using "ix". 

<hr>

## Function application

<b>Evaluating a function taking each element of a Series as argument</b>

To apply any <i>one-argument</i> function to each element of a Series, use .map

In [251]:
def signed_shifted_log(x):
    return np.sign(x) * np.log(np.abs(x) + 1)

def classify(x):
    value = signed_shifted_log(x)
    if value < 1:
        return "Tier A"
    elif value < 2:
        return "Tier B"
    else:
        return "Tier C"

In [252]:
s.map(signed_shifted_log)

a    2.036785
b    1.022416
c    1.379231
d    0.929286
a    2.024599
dtype: float64

In [253]:
s.map(classify)

a    Tier C
b    Tier B
c    Tier B
d    Tier A
a    Tier C
dtype: object

<b>Evaluating a function taking each column of a DataFrame as an argument</b>

In [270]:
def midpoint(x): # x is supposed to be an array of numerical values
    maximum = np.max(x)
    minimum = np.min(x)
    return (maximum - minimum)/2

The <i>apply</i> method returns a Series containing the evaluated value for each column

In [273]:
df.apply(midpoint)

col1    2.363260
col2    1.225268
col3    3.699168
col4    2.420995
dtype: float64

<b>Evaluating a function taking each entry of a DataFrame as argument</b>

The <i>applymap</i> selects each entry of DataFrame and applies a function to it.

In [275]:
df.applymap(classify)

col1    col2    col3    col4
a  Tier C  Tier C  Tier B  Tier C
b  Tier B  Tier C  Tier C  Tier B
c  Tier B  Tier C  Tier C  Tier C
d  Tier B  Tier C  Tier C  Tier C
a  Tier C  Tier C  Tier A  Tier C

￼<hr>

# Split-Apply-Combine

<img src = "split_apply_combine.png", width = 400>

In [330]:
baby_names = pd.read_csv("baby_names.csv")

In [331]:
baby_names.head()

name gender  count  year
0       Mary      F   7065  1880
1       Anna      F   2604  1880
2       Emma      F   2003  1880
3  Elizabeth      F   1939  1880
4     Minnie      F   1746  1880

In [332]:
baby_names.shape

(1825433, 4)

## <b>Aggregation</b>

<b>Applying a function to aggregate a group defined by one column</b>

In [438]:
baby_names[baby_names["name"] == "Vitor"].head()

name gender  count  year  cumulative
1274502  Vitor      M      6  1997           6
1299506  Vitor      M      9  1998          15
1328774  Vitor      M      8  1999          23
1449335  Vitor      M      9  2003          32
1512108  Vitor      M     12  2005          44

Let's find out the total name count across the years.

In [362]:
name_count = baby_names.groupby(by = ["name"])["count"].agg(np.sum)
name_count.head()

name
Aaban         72
Aabha         21
Aabid          5
Aabriella     10
Aadam        196
Name: count, dtype: int64

Note that now "name" is the index, and the total sum of "count" per name is the new value.

<b>Exercise</b> Find the smallest year associated with each first baby name.

In [365]:
# Uncomment and complete the next line
#baby_names.groupby(...)[""].agg()

<b>Applying a function to aggregate a group defined more than one column</b>

Let's find out the total name count by name *and* gender.

In [366]:
name_count_by_gender = baby_names.groupby(by = ["name", "gender"])["count"].agg(np.sum)

In [370]:
name_count_by_gender.head(20)

<font color="navy">You can also unstack the names to create a pivot table</b>

In [371]:
name_count = name_count_by_gender.unstack().fillna(0)

In [372]:
name_count[:10000].tail()

gender     F    M
name             
Barkot     6    0
Barksdale  0    5
Barlow     0   82
Barnabas   0  401
Barnaby    0  314

## Transformation

Instead of aggregating and returning <i>one</i> value, the output of <i>transform</i> has the same length as the original input.

Example: keep track of cumulative sum of baby names across the years

In [412]:
baby_names[baby_names["name"] == "Vitor"]#["count"].cumsum()

name gender  count  year  cumulative
1274502  Vitor      M      6  1997           6
1299506  Vitor      M      9  1998          15
1328774  Vitor      M      8  1999          23
1449335  Vitor      M      9  2003          32
1512108  Vitor      M     12  2005          44
1545784  Vitor      M     12  2006          56
1581605  Vitor      M     10  2007          66
1616568  Vitor      M     10  2008          76
1655456  Vitor      M      6  2009          82
1687010  Vitor      M      8  2010          90
1719880  Vitor      M      9  2011          99
1755639  Vitor      M      7  2012         106
1792274  Vitor      M      5  2013         111
1819476  Vitor      M     10  2014         121

This is useful when creating a new column

In [393]:
baby_names["cumulative"] = baby_names.groupby(by = ["name"])["count"].transform(np.cumsum)

## <font color = "navy"> Filtration</font>

The method <i>filter</i> applies a boolean function to a group, and returns the rows of that group only if the function evaluates to True.

In [464]:
def is_rare(x):
    return np.sum(x["count"]) < 20

In [465]:
rare_names = baby_names.ix[:500,:].groupby(by = ["name"]).filter(is_rare)

In [467]:
rare_names.head()

name gender  count  year  cumulative
374  Antoinette      F     19  1880          19
375  Clementine      F     19  1880          19
376      Edythe      F     19  1880          19
377   Harriette      F     19  1880          19
378      Libbie      F     19  1880          19

<hr>

# Combining DataFrames

<b>Merging on columns</b>

In [563]:
dt1 = {"name": ['Bernie', 'Sanders', 'Donald', 'Trump'], "value1":np.random.uniform(size =(4,))}
dt2 = {"name": ['Bernie', 'Sanders', 'Marco', 'Rubio'], "value2":np.random.uniform(size =(4,))}

In [564]:
df1 = pd.DataFrame(dt1)
df2 = pd.DataFrame(dt2)

In [565]:
df1

name    value1
0   Bernie  0.106293
1  Sanders  0.091624
2   Donald  0.036304
3    Trump  0.323085

In [566]:
df2

name    value2
0   Bernie  0.767844
1  Sanders  0.478669
2    Marco  0.979002
3    Rubio  0.071727

## Merge

The function <i>merge</i> combines DataFrames using a specified *column*.

In [562]:
merged = pd.merge(df1, df2, on = "name", how = "inner")
merged

name    value1    value2
0   Bernie  0.311335  0.911761
1   Bernie  0.311335  0.573102
2   Bernie  0.311335  0.259590
3   Bernie  0.523156  0.911761
4   Bernie  0.523156  0.573102
5   Bernie  0.523156  0.259590
6  Sanders  0.211870  0.413802

## Concat

My favorite way of combining DataFrames. It glues together a *list* (or tuple) of datasets by either stacking them on top of each other, or combining them by their indices.

dt1 = {"name": ['Bernie', 'Sanders', 'Donald', 'Trump'], "value1":np.random.uniform(size =(4,))}
dt2 = {"name": ['Bernie', 'Sanders', 'Marco', 'Rubio'], "value2":np.random.uniform(size =(4,))}
dt3 = {"name": ['Bernie', 'Sanders', 'Marco', 'Trump'], "value3":np.random.uniform(size =(4,))}

In [581]:
df1 = pd.DataFrame(dt1)
df2 = pd.DataFrame(dt2)
df3 = pd.DataFrame(dt3)

In [582]:
df1 = df1.set_index("name")
df2 = df2.set_index("name")
df3 = df3.set_index("name")

In [592]:
pd.concat([df1, df2, df3], axis = 1)

value1    value2    value3
Bernie   0.857105  0.999659  0.619943
Donald   0.428920       NaN       NaN
Marco         NaN  0.672782  0.117680
Rubio         NaN  0.611643       NaN
Sanders  0.277410  0.157756  0.402739
Trump    0.359088       NaN  0.414151

<hr>

# More information

This 2012 video is a bit outdated, but it still worth going through. Wes McKinney (creator of Pandas) uses Python 2.7, and the syntax has evolved just a little, so you might not be able to reproduce examples exactly. But it is not hard to figure out what you need to change.


https://www.youtube.com/watch?v=w26x-z-BdWQ


Wes McKinney's "Python for Data Analysis" is also quite interesting, but most of the information is available in the documentation website: 

http://pandas.pydata.org/pandas-docs/stable/

Be sure to check out, at the very least, "10 Minutes To Pandas".

I also recommend the Visualization chapter: http://pandas.pydata.org/pandas-docs/stable/visualization.html.